In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from data_utils.data import CatsAndDogs
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F
from models.classifier import Classifier

In [9]:
BATCH_SIZE = 128
ETA = 1e-4
EPOCHS = 5

In [4]:
def compute_accuracy(ps, labels):
    equality = (torch.round(ps) == labels).float()
    acc = equality.mean()
    return acc

In [5]:
def train_step(context, x, y):
    model = context["model"]
    optimizer = context["optimizer"]
    
    optimizer.zero_grad()
    
    ps = model(x)
    ps = ps.squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    loss.backward()
    optimizer.step()
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [6]:
def eval_step(context, x, y):
    model = context["model"]
    
    ps = model(x)
    ps = ps.squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [11]:
def train():
    trainset = CatsAndDogs(mode="train", distortion=True, crop=True)
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    
    testset = CatsAndDogs(mode="test")
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    model = Classifier().cuda()
    optimizer = optim.Adam(model.parameters())
    
    context = {
        "model": model,
        "optimizer": optimizer
    }
    
    for e in range(EPOCHS):
        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0
        
        model.train()
        
        for x, y in train_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            loss, acc = train_step(context, x, y)
            train_loss += loss
            train_acc += acc
            
        model.eval()
            
        for x, y in test_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            with torch.no_grad():
                loss, acc = eval_step(context, x, y)
                test_loss += loss
                test_acc += acc
                
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
        if (e+1)%1 == 0:
            print(f"Epochs {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}, train acc: {train_acc:.4f}")
            print(f"Test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")

In [12]:
train()

Epochs 1/5
Train loss: 0.13224773, train acc: 0.9447
Test loss: 0.19956845, test acc: 0.9223
Epochs 2/5
Train loss: 0.09417461, train acc: 0.9618
Test loss: 0.08849475, test acc: 0.9657
Epochs 3/5
Train loss: 0.07567671, train acc: 0.9701
Test loss: 0.07304023, test acc: 0.9709
Epochs 4/5
Train loss: 0.06283667, train acc: 0.9755
Test loss: 0.09545016, test acc: 0.9638
Epochs 5/5
Train loss: 0.05446789, train acc: 0.9786
Test loss: 0.08275525, test acc: 0.9691
